# Extract Data
Extracting Amenities Data via Google Map API. You need include `GOOGLE_MAPS_API_KEY` in `.env`.

In [ ]:
import os
import requests
import pandas as pd
import time
from dotenv import load_dotenv

In [ ]:
# Load .env
load_dotenv()
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')

location = '41.7943,-87.5907'  # Center of Hyde Park
radius = 3000
place_types = [
    'grocery_or_supermarket', 'cafe', 'library', 'book_store', 'university',
    'hospital', 'pharmacy', 'doctor', 'dentist', 'park', 'gym', 'bank', 'atm',
    'post_office', 'restaurant', 'meal_takeaway', 'meal_delivery',
    'hair_care', 'beauty_salon', 'convenience_store', 'drugstore'
]

all_places = []

def fetch_places(place_type):
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params = {
        'location': location,
        'radius': radius,
        'type': place_type,
        'key': API_KEY
    }
    results = []

    while True:
        res = requests.get(url, params=params).json()
        results.extend(res.get('results', []))

        next_token = res.get('next_page_token')
        if not next_token:
            break
        time.sleep(2)
        params = {'pagetoken': next_token, 'key': API_KEY}

    return results

for place_type in place_types:
    print(f"Fetching: {place_type}")
    places = fetch_places(place_type)
    for p in places:
        all_places.append({
            'type': place_type,
            'name': p.get('name'),
            'address': p.get('vicinity'),
            'lat': p['geometry']['location']['lat'],
            'lng': p['geometry']['location']['lng'],
            'rating': p.get('rating'),
            'user_ratings_total': p.get('user_ratings_total')
        })
    time.sleep(1)

df = pd.DataFrame(all_places)
df.to_csv('../data/hyde_park_amenities.csv', index=False)
print("Saved as hyde_park_amenities.csv")

# Load Data
Creating table and ingest data to db.

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine


In [3]:
# load .env
load_dotenv()
db_url = os.getenv("DATABASE_URL")

# Create SQLAlchemy engine
engine = create_engine(db_url)

# Read CSV
df = pd.read_csv("../data/hyde_park_amenities.csv")

# Create table and ingest the data to thetable
df.to_sql("apt_app_amenity_raw", engine, if_exists='replace', index=False)


752

### Check DB

In [4]:
# CSV
df = pd.read_csv("../data/hyde_park_amenities.csv")
print("CSV counts:", len(df))

# DB
db_count = pd.read_sql("SELECT COUNT(*) AS count FROM apt_app_amenity_raw", engine).iloc[0, 0]
print("DB counts:", db_count)

# head
df_preview = pd.read_sql("SELECT * FROM apt_app_amenity_raw LIMIT 5", engine)
print(df_preview)


CSV counts: 752
DB counts: 752
                     type                      name  \
0  grocery_or_supermarket  Hyde Park Produce Market   
1  grocery_or_supermarket                      ALDI   
2  grocery_or_supermarket                        bp   
3  grocery_or_supermarket        Whole Foods Market   
4  grocery_or_supermarket   One Stop Food & Liquors   

                                    address        lat        lng  rating  \
0            1226 East 53rd Street, Chicago  41.800011 -87.595481     4.5   
1  6621 South Cottage Grove Avenue, Chicago  41.774518 -87.605391     4.2   
2     6701 South Jeffery Boulevard, Chicago  41.773106 -87.576174     3.3   
3      5118 South Lake Park Avenue, Chicago  41.801803 -87.588083     4.2   
4      4301 South Lake Park Avenue, Chicago  41.816212 -87.598257     4.1   

   user_ratings_total  
0              1084.0  
1              1538.0  
2                17.0  
3              1835.0  
4              1493.0  


# Transform Data
To transform raw data into django data, you need to run `scripts/amenity_transform.py`.